## Implementation using datetime's tzinfo method
One needs to:
* create a specific class from tzinfo abstract subclass
* such specific class should be created for every country/regionspecifically (not necessarily for every timezone within one country)

In [104]:
import datetime as dte

ZERO = dte.timedelta(0)
HOUR = dte.timedelta(hours=1)

def first_sunday_on_or_after(dt):
    days_to_go = 6 - dt.weekday()
    if days_to_go:
        dt += dte.timedelta(days_to_go)
    return dt


# US DST Rules
#
# This is a simplified (i.e., wrong for a few cases) set of rules for US
# DST start and end times. For a complete and up-to-date set of DST rules
# and timezone definitions, visit the Olson Database (or try pytz):
# http://www.twinsun.com/tz/tz-link.htm
# http://sourceforge.net/projects/pytz/ (might not be up-to-date)
#
# In the US, since 2007, DST starts at 2am (standard time) on the second
# Sunday in March, which is the first Sunday on or after Mar 8.
DSTSTART_2007 = dte.datetime(1, 3, 8, 2)
# print(DSTSTART_2007.strftime("%Y%m%d %H:%M:%S"))
# and ends at 2am (DST time; 1am standard time) on the first Sunday of Nov.
DSTEND_2007 = dte.datetime(1, 11, 1, 1)
# From 1987 to 2006, DST used to start at 2am (standard time) on the first
# Sunday in April and to end at 2am (DST time; 1am standard time) on the last
# Sunday of October, which is the first Sunday on or after Oct 25.
DSTSTART_1987_2006 = dte.datetime(1, 4, 1, 2)
DSTEND_1987_2006 = dte.datetime(1, 10, 25, 1)
# From 1967 to 1986, DST used to start at 2am (standard time) on the last
# Sunday in April (the one on or after April 24) and to end at 2am (DST time;
# 1am standard time) on the last Sunday of October, which is the first Sunday
# on or after Oct 25.
DSTSTART_1967_1986 = dte.datetime(1, 4, 24, 2)
DSTEND_1967_1986 = DSTEND_1987_2006

class USTimeZone(dt.tzinfo):

    def __init__(self, hours, reprname, stdname, dstname):
        self.stdoffset = dte.timedelta(hours=hours)
        self.reprname = reprname
        self.stdname = stdname
        self.dstname = dstname

    def __repr__(self):
        return self.reprname

    def tzname(self, dt):
        if self.dst(dt):
            return self.dstname
        else:
            return self.stdname

    def utcoffset(self, dt):
        return self.stdoffset + self.dst(dt)

    def dst(self, dt):
        if dt is None or dt.tzinfo is None:
            # An exception may be sensible here, in one or both cases.
            # It depends on how you want to treat them.  The default
            # fromutc() implementation (called by the default astimezone()
            # implementation) passes a datetime with dt.tzinfo is self.
            return ZERO
        assert dt.tzinfo is self

        # Find start and end times for US DST. For years before 1967, return
        # ZERO for no DST.
        if 2006 < dt.year:
            dststart, dstend = DSTSTART_2007, DSTEND_2007
        elif 1986 < dt.year < 2007:
            dststart, dstend = DSTSTART_1987_2006, DSTEND_1987_2006
        elif 1966 < dt.year < 1987:
            dststart, dstend = DSTSTART_1967_1986, DSTEND_1967_1986
        else:
            return ZERO

        start = first_sunday_on_or_after(dststart.replace(year=dt.year))
        end = first_sunday_on_or_after(dstend.replace(year=dt.year))

        # Can't compare naive to aware objects, so strip the timezone from
        # dt first.
        if start <= dt.replace(tzinfo=None) < end:
            return HOUR
        else:
            return ZERO

Eastern  = USTimeZone(-5, "Eastern",  "EST", "EDT")
Central  = USTimeZone(-6, "Central",  "CST", "CDT")
Mountain = USTimeZone(-7, "Mountain", "MST", "MDT")
Pacific  = USTimeZone(-8, "Pacific",  "PST", "PDT")
London = USTimeZone(0, "London",  "GMT", "BST")

my_date_ET = dt.datetime(2018,7,16,10,30,0,tzinfo=Eastern)
print(my_date_ET)
print(type(my_date_ET))
my_date_PT = my_date_ET.astimezone(Pacific)
print(my_date_PT)
print(type(my_date_PT))
my_date_LN = my_date_ET.astimezone(London)
print(my_date_LN)
print(type(my_date_LN))
utc_time = my_date_ET - dt.datetime.utcoffset(my_date_ET)
print(utc_time)

2018-07-16 10:30:00-04:00
<class 'datetime.datetime'>
2018-07-16 07:30:00-07:00
<class 'datetime.datetime'>
2018-07-16 15:30:00+01:00
<class 'datetime.datetime'>
2018-07-16 14:30:00-04:00


## Implementation using Panda's tz_localize and tz_covert methods
One needs to:
* convert datetime objects to pandas.series object

In [118]:
import time
import datetime as dt
import pandas as pd

times = [dt.time(10,0,0), dt.time(10,30,0), dt.time(11,0,0), dt.time(11,30,0), dt.time(12,0,0), dt.time(12,30,0), 
         dt.time(13,0,0), dt.time(13,30,0), dt.time(14,0,0), dt.time(14,30,0), dt.time(15,0,0), dt.time(15,30,0), 
         dt.time(16,0,0)]

start_dt = "05/27/2018"
end_dt = "09/19/2018"
Trading_Dates = pd.bdate_range(start_dt, end_dt, freq="B")

x = 0

for trading_date in Trading_Dates:
    x +=1
    #print(trading_date.strftime("%Y%m%d %H:%M:%S"))
    for time in times:
        date_plus_time = dt.datetime.combine(trading_date,time)
        print(date_plus_time, date_plus_time.tzinfo, type(date_plus_time))
        pd_date_plus_time = pd.to_datetime(date_plus_time)
        #date_plus_time_tz = pd.tz_localize(date_plus_time)
        print(pd_date_plus_time, pd_date_plus_time.tzinfo, type(pd_date_plus_time))
        pd_date_plus_time_tz = pd_date_plus_time.tz_localize(tz="US/Eastern")
        print(pd_date_plus_time_tz, pd_date_plus_time_tz.tzinfo, type(pd_date_plus_time_tz))
        pd_date_plus_time_tz_utc = pd_date_plus_time_tz.tz_convert(tz="UTC")
        print(pd_date_plus_time_tz_utc, pd_date_plus_time_tz_utc.tzinfo, type(pd_date_plus_time_tz_utc))
        print(pd_date_plus_time_tz_utc.strftime("%Y%m%d %H:%M:%S"))
        print("\n")
    if x == 2:
        break
# end_trading_time = dt.datetime(10,0,0)
# end_trading_time.strftime("%H:%M:%S")

# tm = dt.datetime(2018,9,15,10,0,0)
# print(tm.strftime("%H:%M:%S"))
# print(tm.tzinfo)
# UTC_time = end_trading_time - dt.timedelta(minutes=30)
# UTC_time

2018-05-28 10:00:00 None <class 'datetime.datetime'>
2018-05-28 10:00:00 None <class 'pandas._libs.tslibs.timestamps.Timestamp'>
2018-05-28 10:00:00-04:00 US/Eastern <class 'pandas._libs.tslibs.timestamps.Timestamp'>
2018-05-28 14:00:00+00:00 UTC <class 'pandas._libs.tslibs.timestamps.Timestamp'>
20180528 14:00:00


2018-05-28 10:30:00 None <class 'datetime.datetime'>
2018-05-28 10:30:00 None <class 'pandas._libs.tslibs.timestamps.Timestamp'>
2018-05-28 10:30:00-04:00 US/Eastern <class 'pandas._libs.tslibs.timestamps.Timestamp'>
2018-05-28 14:30:00+00:00 UTC <class 'pandas._libs.tslibs.timestamps.Timestamp'>
20180528 14:30:00


2018-05-28 11:00:00 None <class 'datetime.datetime'>
2018-05-28 11:00:00 None <class 'pandas._libs.tslibs.timestamps.Timestamp'>
2018-05-28 11:00:00-04:00 US/Eastern <class 'pandas._libs.tslibs.timestamps.Timestamp'>
2018-05-28 15:00:00+00:00 UTC <class 'pandas._libs.tslibs.timestamps.Timestamp'>
20180528 15:00:00


2018-05-28 11:30:00 None <class 'datetime.date

## All Olson's timezones
* for above Pandas implementation using tz_localize and tz_convert

In [80]:
import pytz
pytz.all_timezones

['Africa/Abidjan', 'Africa/Accra', 'Africa/Addis_Ababa', 'Africa/Algiers', 'Africa/Asmara', 'Africa/Asmera', 'Africa/Bamako', 'Africa/Bangui', 'Africa/Banjul', 'Africa/Bissau', 'Africa/Blantyre', 'Africa/Brazzaville', 'Africa/Bujumbura', 'Africa/Cairo', 'Africa/Casablanca', 'Africa/Ceuta', 'Africa/Conakry', 'Africa/Dakar', 'Africa/Dar_es_Salaam', 'Africa/Djibouti', 'Africa/Douala', 'Africa/El_Aaiun', 'Africa/Freetown', 'Africa/Gaborone', 'Africa/Harare', 'Africa/Johannesburg', 'Africa/Juba', 'Africa/Kampala', 'Africa/Khartoum', 'Africa/Kigali', 'Africa/Kinshasa', 'Africa/Lagos', 'Africa/Libreville', 'Africa/Lome', 'Africa/Luanda', 'Africa/Lubumbashi', 'Africa/Lusaka', 'Africa/Malabo', 'Africa/Maputo', 'Africa/Maseru', 'Africa/Mbabane', 'Africa/Mogadishu', 'Africa/Monrovia', 'Africa/Nairobi', 'Africa/Ndjamena', 'Africa/Niamey', 'Africa/Nouakchott', 'Africa/Ouagadougou', 'Africa/Porto-Novo', 'Africa/Sao_Tome', 'Africa/Timbuktu', 'Africa/Tripoli', 'Africa/Tunis', 'Africa/Windhoek', 'Ameri